In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/food-nutrition-dataset/food.csv
/kaggle/input/food-nutrition-dataset/food1.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import openai
import requests
from PIL import Image
from io import BytesIO
import torch
from transformers import pipeline, BlipProcessor, BlipForConditionalGeneration


2025-05-30 08:57:44.621129: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748595464.644839     121 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748595464.653020     121 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppress TensorFlow logs


In [4]:
food_data = pd.read_csv('/kaggle/input/food-nutrition-dataset/food.csv')  # update path as needed
food_data.head()


,Category,Description,Nutrient Data Bank Number,Data.Alpha Carotene,Data.Ash,Data.Beta Carotene,Data.Beta Cryptoxanthin,Data.Carbohydrate,Data.Cholesterol,Data.Choline,...,Data.Major Minerals.Potassium,Data.Major Minerals.Sodium,Data.Major Minerals.Zinc,Data.Vitamins.Vitamin A - IU,Data.Vitamins.Vitamin A - RAE,Data.Vitamins.Vitamin B12,Data.Vitamins.Vitamin B6,Data.Vitamins.Vitamin C,Data.Vitamins.Vitamin E,Data.Vitamins.Vitamin K
0,BUTTER,"BUTTER,WITH SALT",1001,0,2.11,158,0,0.06,215,19,...,24,576,0.09,2499,684,0.17,0.003,0.0,2.32,7.0
1,BUTTER,"BUTTER,WHIPPED,WITH SALT",1002,0,2.11,158,0,0.06,219,19,...,26,827,0.05,2499,684,0.13,0.003,0.0,2.32,7.0
2,BUTTER OIL,"BUTTER OIL,ANHYDROUS",1003,0,0.00,193,0,0.00,256,22,...,5,2,0.01,3069,840,0.01,0.001,0.0,2.80,8.6
3,CHEESE,"CHEESE,BLUE",1004,0,5.11,74,0,2.34,75,15,...,256,1395,2.66,763,198,1.22,0.166,0.0,0.25,2.4
4,CHEESE,"CHEESE,BRICK",1005,0,3.18,76,0,2.79,94,15,...,136,560,2.60,1080,292,1.26,0.065,0.0,0.26,2.5


In [5]:
from transformers import pipeline
nutrition_bot = pipeline("text-generation", model="gpt2")

def ask_bot(query):
    return nutrition_bot(query, max_length=100)[0]['generated_text']


Device set to use cuda:0


In [6]:
from transformers import BlipProcessor, BlipForConditionalGeneration

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

def caption_image(img_url):
    raw = requests.get(img_url).content
    image = Image.open(BytesIO(raw)).convert('RGB')
    inputs = processor(image, return_tensors="pt")
    out = model.generate(**inputs)
    return processor.decode(out[0], skip_special_tokens=True)


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [7]:
def generate_meal_plan(goal, allergies, preferences):
    # Filter from food_data
    filtered = food_data[
        ~food_data['Ingredients'].str.contains('|'.join(allergies), case=False, na=False)
    ]
    # Filter by goal & preference logic (simplified for now)
    sample_plan = filtered.sample(3)  # replace with logic later
    return sample_plan[['Food', 'Calories', 'Protein', 'Fat', 'Carbs']]


In [8]:
def explain_food_choice(food_item):
    query = f"Why is {food_item} considered healthy?"
    return ask_bot(query)


In [9]:
import seaborn as sns

def visualize_nutrition(meal_plan):
    meal_plan.set_index('Food')[['Calories', 'Protein', 'Fat', 'Carbs']].plot(kind='bar', figsize=(10,5))
    plt.title('Meal Plan Nutritional Breakdown')
    plt.ylabel('Amount')
    plt.show()


In [12]:
# Sample input + response pipeline

def nutrition_assistant_interface():
    print("👋 Welcome to the Smartest AI Nutrition Assistant!")
    user_input = input("Ask me anything about your diet, health goals, or a food item:\n> ")

    # Generate response (you can plug this into your GPT or rules-based model)
    response = ask_bot(user_input)  # Replace with your function (e.g., GPT-based)
    
    print("\n🤖 Assistant says:")
    print(response)

# Run the interface
nutrition_assistant_interface()


👋 Welcome to the Smartest AI Nutrition Assistant!


Ask me anything about your diet, health goals, or a food item:
>  Create a high-protein meal plan for muscle gain.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



🤖 Assistant says:
Create a high-protein meal plan for muscle gain.

When you start out, make sure you have a healthy diet but not so far that the protein that you get comes flooding into your bloodstreams.

It is this blood insulin that gets a little out of balance, which often leads to issues with muscle growth.

Some of those common blood markers:

Acute-onset diabetes

Liver enzymes and glycosylated staphylates

V


In [24]:
# Sample input + response pipeline

def nutrition_assistant_interface():
    print("👋 Welcome to the Smartest AI Nutrition Assistant!")
    user_input = input("Ask me anything about your diet, health goals, or a food item:\n> ")

    # Generate response (you can plug this into your GPT or rules-based model)
    response = ask_bot(user_input)  # Replace with your function (e.g., GPT-based)
    
    print("\n🤖 Assistant says:")
    print(response)

# Run the interface
nutrition_assistant_interface()


👋 Welcome to the Smartest AI Nutrition Assistant!


Ask me anything about your diet, health goals, or a food item:
>  Why are almonds good for weight loss?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



🤖 Assistant says:
Why are almonds good for weight loss?

They're good for eating, they get you to eat a lot, and you don't make other people do anything, you don't think about their diet. What they're good for is you get the right type of sugar, a carbohydrate, so that you're not putting this into a food system that's not digestible when you eat it; instead, they give you a way to build carbohydrate to get you a fuller body.

What


In [31]:
# Example image captioning test
img_url = "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTI66XDnyYIMaRN8s54vsdepiNLEPyHlYIKTA&s"
print("Detected Food:", caption_image(img_url))


Detected Food: a plate of steak, aspara and potatoes
